# Hello World - Spark


https://spark.apache.org/docs/latest/api/python/user_guide/index.html

In [1]:
import os

os.environ['SPARK_HOME'] = 'C:\spark\spark-3.1.2-bin-hadoop2.7'
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jre1.8.0_361'
os.environ['HADOOP_HOME'] = 'C:\spark\spark-3.2.1-bin-hadoop2.7\hadoop'

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.functions import *



In [2]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName("Iniciando com Spark")\
    .config('spark.ui.port', '4050')\
    .getOrCreate()

spark

# Importando dados

In [3]:
seattle_library = spark.read.option("header","true").csv('Checkouts_by_Title.csv')
seattle_library.show()

+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+
|UsageClass|CheckoutType|MaterialType|CheckoutYear|CheckoutMonth|Checkouts|               Title|ISBN|             Creator|            Subjects|           Publisher|PublicationYear|
+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+
|  Physical|     Horizon|        BOOK|        2007|           12|        1|Opposite the Cros...|null|       Haymon, S. T.|Haymon S T Childh...| St. Martin's Press,|         c1988.|
|  Physical|     Horizon|        BOOK|        2007|           12|        1| Power of persuasion|null|                null|Love stories, Pol...|                null|           null|
|  Physical|     Horizon|        BOOK|        2007|           12|        1|Cinema nirvana en...

### Adicionando ID_Material

In [4]:
seattle_library = seattle_library\
                    .withColumn("ID_Material",
                                sha2(concat_ws("|","UsageClass","MaterialType", "Title", "ISBN", "Creator", "Subjects", "Publisher", "PublicationYear"),256))

seattle_library.show(10,False)

+----------+------------+------------+------------+-------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+---------------+----------------------------------------------------------------+
|UsageClass|CheckoutType|MaterialType|CheckoutYear|CheckoutMonth|Checkouts|Title                                                                                                                                                                                             

In [5]:
seattle_library.ID_Material = sha2(seattle_library.ID_Material,256)

In [6]:
seattle_library.show(10)

+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+--------------------+
|UsageClass|CheckoutType|MaterialType|CheckoutYear|CheckoutMonth|Checkouts|               Title|ISBN|             Creator|            Subjects|           Publisher|PublicationYear|         ID_Material|
+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+--------------------+
|  Physical|     Horizon|        BOOK|        2007|           12|        1|Opposite the Cros...|null|       Haymon, S. T.|Haymon S T Childh...| St. Martin's Press,|         c1988.|435cb8e8e17d73133...|
|  Physical|     Horizon|        BOOK|        2007|           12|        1| Power of persuasion|null|                null|Love stories, Pol...|                null|           null|8dd007d2173f

### Adicionando ID_Checkout

In [7]:
seattle_library = seattle_library.withColumn("ID_Checkout", monotonically_increasing_id())

seattle_library.show(10)

+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+--------------------+-----------+
|UsageClass|CheckoutType|MaterialType|CheckoutYear|CheckoutMonth|Checkouts|               Title|ISBN|             Creator|            Subjects|           Publisher|PublicationYear|         ID_Material|ID_Checkout|
+----------+------------+------------+------------+-------------+---------+--------------------+----+--------------------+--------------------+--------------------+---------------+--------------------+-----------+
|  Physical|     Horizon|        BOOK|        2007|           12|        1|Opposite the Cros...|null|       Haymon, S. T.|Haymon S T Childh...| St. Martin's Press,|         c1988.|435cb8e8e17d73133...|          0|
|  Physical|     Horizon|        BOOK|        2007|           12|        1| Power of persuasion|null|                null|Love stories, Pol...| 

In [8]:
seattle_library.printSchema()


root
 |-- UsageClass: string (nullable = true)
 |-- CheckoutType: string (nullable = true)
 |-- MaterialType: string (nullable = true)
 |-- CheckoutYear: string (nullable = true)
 |-- CheckoutMonth: string (nullable = true)
 |-- Checkouts: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Creator: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- PublicationYear: string (nullable = true)
 |-- ID_Material: string (nullable = true)
 |-- ID_Checkout: long (nullable = false)



### Contando quantidade de valores nulos por coluna

In [9]:
#seattle_library.select([count(when(isnull(c),1)).alias(c) for c in seattle_library.columns]).show()

In [10]:
#seattle_library.select('*')\
 #   .orderBy('Title',asceding=False)\
  #  .show(10)


In [11]:
#seattle_library.select('CheckoutYear','Checkouts','Title')\
 #   .groupBy('CheckoutYear','Title')\
  #  .count()\
   # .orderBy('count',asceding=False)\
    #.show()

## Selecionando Dados

## Tabela Material

In [12]:
#Material
seattle_library_material = seattle_library\
            .select('ID_Material','UsageClass','Title','ISBN','Publisher','PublicationYear')

seattle_library_material.show(5)

+--------------------+----------+--------------------+----+-------------------+---------------+
|         ID_Material|UsageClass|               Title|ISBN|          Publisher|PublicationYear|
+--------------------+----------+--------------------+----+-------------------+---------------+
|435cb8e8e17d73133...|  Physical|Opposite the Cros...|null|St. Martin's Press,|         c1988.|
|8dd007d2173f57e16...|  Physical| Power of persuasion|null|               null|           null|
|6bf3ac5e1d619e457...|  Physical|Cinema nirvana en...|null|               null|           null|
|401b0417fcbc904b5...|  Physical| Nepodvedennye itogi|null|               null|           null|
|95686048521296ba2...|  Physical|Xu Zhian He Yunsh...|null|               null|           null|
+--------------------+----------+--------------------+----+-------------------+---------------+
only showing top 5 rows



In [13]:
#seattle_library_material.select('Publisher').distinct().collect()

## Tabela Checkout Month

In [14]:
#Checkouts_Month
seattle_library_checkouts_month = seattle_library\
            .select('ID_Checkout','CheckoutType','CheckoutYear','CheckoutMonth','Checkouts','ID_Material')

seattle_library_checkouts_month.show(5,False)

+-----------+------------+------------+-------------+---------+----------------------------------------------------------------+
|ID_Checkout|CheckoutType|CheckoutYear|CheckoutMonth|Checkouts|ID_Material                                                     |
+-----------+------------+------------+-------------+---------+----------------------------------------------------------------+
|0          |Horizon     |2007        |12           |1        |435cb8e8e17d73133e338a535e72f30f25ea15179691dc04b4e492447e2f2bdf|
|1          |Horizon     |2007        |12           |1        |8dd007d2173f57e16e30a050fb9ce05996ff7b17f3422db8bf3928faada2d47b|
|2          |Horizon     |2007        |12           |1        |6bf3ac5e1d619e4579ee77f0c52e6a41eda29ddda1b9be8f72c2721854164a2d|
|3          |Horizon     |2007        |12           |2        |401b0417fcbc904b5e8e03d0ef23b4ce883317b8cfb83681d3664ff5a3758b49|
|4          |Horizon     |2007        |12           |4        |95686048521296ba224ac8664ccf7c466c

In [15]:
#seattle_library_checkouts_month.filter("ID_Material == '435cb8e8e17d73133e338a535e72f30f25ea15179691dc04b4e492447e2f2bdf'").show()

## Tabela Material Type

In [16]:
#Material_Type
seattle_library_material_type = seattle_library\
        .select('MaterialType','ID_Material')

seattle_library_material_type.show(5,False)

+------------+----------------------------------------------------------------+
|MaterialType|ID_Material                                                     |
+------------+----------------------------------------------------------------+
|BOOK        |435cb8e8e17d73133e338a535e72f30f25ea15179691dc04b4e492447e2f2bdf|
|BOOK        |8dd007d2173f57e16e30a050fb9ce05996ff7b17f3422db8bf3928faada2d47b|
|BOOK        |6bf3ac5e1d619e4579ee77f0c52e6a41eda29ddda1b9be8f72c2721854164a2d|
|BOOK        |401b0417fcbc904b5e8e03d0ef23b4ce883317b8cfb83681d3664ff5a3758b49|
|VIDEODISC   |95686048521296ba224ac8664ccf7c466ccabfdcf92cba9c8d526b3ddc986525|
+------------+----------------------------------------------------------------+
only showing top 5 rows



In [17]:
split_col = split(seattle_library_material_type['MaterialType'], ',')

df2 = seattle_library_material_type\
    .withColumn('material_type_0', split_col.getItem(0)) \
    .withColumn('material_type_1', split_col.getItem(1)) \
    .withColumn('material_type_2', split_col.getItem(2)) \
    .withColumn('material_type_3', split_col.getItem(3)) \
    .withColumn('material_type_4', split_col.getItem(4)) \
    .withColumn('material_type_5', split_col.getItem(5))
df2.show(10)

+------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|MaterialType|         ID_Material|material_type_0|material_type_1|material_type_2|material_type_3|material_type_4|material_type_5|
+------------+--------------------+---------------+---------------+---------------+---------------+---------------+---------------+
|        BOOK|435cb8e8e17d73133...|           BOOK|           null|           null|           null|           null|           null|
|        BOOK|8dd007d2173f57e16...|           BOOK|           null|           null|           null|           null|           null|
|        BOOK|6bf3ac5e1d619e457...|           BOOK|           null|           null|           null|           null|           null|
|        BOOK|401b0417fcbc904b5...|           BOOK|           null|           null|           null|           null|           null|
|   VIDEODISC|95686048521296ba2...|      VIDEODISC|           null|         

In [18]:
def melt(df, id_vars, value_vars, var_name, value_name):
    # Lista de colunas não pivoteadas
    id_expr = [col(c) for c in id_vars]

    # Cria uma lista de structs das colunas pivotadas
    value_cols = [struct(lit(c).alias(var_name), col(c).alias(value_name)) for c in value_vars]

    # Cria um array com todas as colunas pivotadas
    expr = array(*value_cols).alias("vars")

    # Transforma as colunas pivotadas em linhas
    df = df.select(*id_expr + [expr]).withColumn("vars", explode("vars"))

    # Separa as colunas variável e valor
    df = df.select(*id_expr + [col("vars." + var_name), col("vars." + value_name)])

    return df

In [19]:
melted_df = melt(df2,
                ["ID_Material"],
                ['material_type_0', 'material_type_1','material_type_2','material_type_3','material_type_4','material_type_5'],
                 "coluna", "MaterialType")

melted_df = melted_df.dropna()
melted_df = melted_df.select('ID_Material','MaterialType')
melted_df.show(10,False)

+----------------------------------------------------------------+------------+
|ID_Material                                                     |MaterialType|
+----------------------------------------------------------------+------------+
|435cb8e8e17d73133e338a535e72f30f25ea15179691dc04b4e492447e2f2bdf|BOOK        |
|8dd007d2173f57e16e30a050fb9ce05996ff7b17f3422db8bf3928faada2d47b|BOOK        |
|6bf3ac5e1d619e4579ee77f0c52e6a41eda29ddda1b9be8f72c2721854164a2d|BOOK        |
|401b0417fcbc904b5e8e03d0ef23b4ce883317b8cfb83681d3664ff5a3758b49|BOOK        |
|95686048521296ba224ac8664ccf7c466ccabfdcf92cba9c8d526b3ddc986525|VIDEODISC   |
|e2294ea01cff8690a86fd3ecf0250f929c9997cd11b5c39df92b71e2a65b4058|BOOK        |
|6012d180ef2cabce932c8b0c46826947bb9e55087b69303af81cc4f6bfc3cae0|SOUNDDISC   |
|32d67b645caf07ed72266a67aea160c74e9b6c0833427f2439f3378109bf17d3|BOOK        |
|148f940fa46e290b33c0041480acef7174d111a8fb73bf6f2c8b344849b54c84|BOOK        |
|6922005b309323114c8bb2cf706fb7a39afc2d2

## Tabela Creator

In [20]:
#Creator
seattle_library_creator = seattle_library\
        .select('Creator','ID_Material')

seattle_library_creator.show(10)

+--------------------+--------------------+
|             Creator|         ID_Material|
+--------------------+--------------------+
|       Haymon, S. T.|435cb8e8e17d73133...|
|                null|8dd007d2173f57e16...|
|                null|6bf3ac5e1d619e457...|
|                null|401b0417fcbc904b5...|
|                null|95686048521296ba2...|
|     Handler, Daniel|e2294ea01cff8690a...|
|                null|6012d180ef2cabce9...|
|Haywood, Carolyn,...|32d67b645caf07ed7...|
|                null|148f940fa46e290b3...|
|                null|6922005b309323114...|
+--------------------+--------------------+
only showing top 10 rows



In [21]:
split_col_creator = split(seattle_library_creator['Creator'], ';')

df3 = seattle_library_creator\
    .withColumn('creator_0', split_col_creator.getItem(0)) \
    .withColumn('creator_1', split_col_creator.getItem(1)) \
    .withColumn('creator_2', split_col_creator.getItem(2)) \
    .withColumn('creator_3', split_col_creator.getItem(3)) \
    .withColumn('creator_4', split_col_creator.getItem(4)) \
    .withColumn('creator_5', split_col_creator.getItem(5))
df3.show(10)

+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|             Creator|         ID_Material|           creator_0|creator_1|creator_2|creator_3|creator_4|creator_5|
+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+
|       Haymon, S. T.|435cb8e8e17d73133...|       Haymon, S. T.|     null|     null|     null|     null|     null|
|                null|8dd007d2173f57e16...|                null|     null|     null|     null|     null|     null|
|                null|6bf3ac5e1d619e457...|                null|     null|     null|     null|     null|     null|
|                null|401b0417fcbc904b5...|                null|     null|     null|     null|     null|     null|
|                null|95686048521296ba2...|                null|     null|     null|     null|     null|     null|
|     Handler, Daniel|e2294ea01cff8690a...|     Handler, Daniel|     null|     n

In [22]:
melted_df = melt(df3,
                ["ID_Material"],
                ['creator_0', 'creator_1','creator_2','creator_3','creator_4','creator_5'],
                 "coluna", "Creator")

melted_df = melted_df.dropna()
melted_df = melted_df.select('ID_Material','Creator')
melted_df.show(10,False)

+----------------------------------------------------------------+-----------------------------+
|ID_Material                                                     |Creator                      |
+----------------------------------------------------------------+-----------------------------+
|435cb8e8e17d73133e338a535e72f30f25ea15179691dc04b4e492447e2f2bdf|Haymon, S. T.                |
|e2294ea01cff8690a86fd3ecf0250f929c9997cd11b5c39df92b71e2a65b4058|Handler, Daniel              |
|32d67b645caf07ed72266a67aea160c74e9b6c0833427f2439f3378109bf17d3|Haywood, Carolyn, 1898-1990  |
|9659aedd14e18df5545a3dd668d7597a4550a2ed6dd6f609ca2cd7ce291e8bc0|Glancey, Jonathan            |
|6ef16d6fea312ccb3aedd24541f3e01c3f0acfd98a0a7ef20ed08aa311a342d8|Russell, Barbara T.          |
|2c10aed35d832af86435b842f07af718ff3073ed9444007a2bda0d65c3187cbc|Cravens, Gwyneth             |
|68c72c597b6dee9b8f74511d7aa9d563b82eccf1cd50bfa691101e883998b02f|Curtis, Sandra R., 1950-     |
|b6d3ff1b27cc8703562a0ba8b3384

## Tabela Subjects

In [27]:
#Subjects
seattle_library_subjects = seattle_library\
    .select('Subjects','ID_Material') \
    .dropDuplicates()

seattle_library_subjects.show(5,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|Subjects                                                                                                                                                                                                                                     |ID_Material                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------+
|Riazanov Eldar, Motion picture producers and directors Soviet Union Biography, Telev

In [28]:
split_col_subjects = split(seattle_library_subjects['Subjects'], ',')

df4 = seattle_library_subjects\
    .withColumn('subject_0', split_col_subjects.getItem(0)) \
    .withColumn('subject_1', split_col_subjects.getItem(1)) \
    .withColumn('subject_4', split_col_subjects.getItem(4)) \
    .withColumn('subject_3', split_col_subjects.getItem(3)) \
    .withColumn('subject_4', split_col_subjects.getItem(4)) \
    .withColumn('subject_5', split_col_subjects.getItem(5)) \
    .withColumn('subject_6', split_col_subjects.getItem(6)) \
    .withColumn('subject_7', split_col_subjects.getItem(7)) \
    .withColumn('subject_8', split_col_subjects.getItem(8)) \
    .withColumn('subject_9', split_col_subjects.getItem(9)) \
    .withColumn('subject_10', split_col_subjects.getItem(10)) \
    .withColumn('subject_11', split_col_subjects.getItem(11)) \
    .withColumn('subject_12', split_col_subjects.getItem(12)) \
    .withColumn('subject_13', split_col_subjects.getItem(13)) \
    .withColumn('subject_14', split_col_subjects.getItem(14)) \
    .withColumn('subject_15', split_col_subjects.getItem(15)) \
    .withColumn('subject_16', split_col_subjects.getItem(16)) \
    .withColumn('subject_17', split_col_subjects.getItem(17)) \
    .withColumn('subject_18', split_col_subjects.getItem(18)) \
    .withColumn('subject_19', split_col_subjects.getItem(19)) \
    .withColumn('subject_20', split_col_subjects.getItem(20)) \
    .withColumn('subject_21', split_col_subjects.getItem(21)) \
    .withColumn('subject_22', split_col_subjects.getItem(22)) \
    .withColumn('subject_23', split_col_subjects.getItem(23)) \
    .withColumn('subject_24', split_col_subjects.getItem(24)) \
    .withColumn('subject_25', split_col_subjects.getItem(25)) \
    .withColumn('subject_26', split_col_subjects.getItem(26)) \
    .withColumn('subject_27', split_col_subjects.getItem(27)) \
    .withColumn('subject_28', split_col_subjects.getItem(28)) \
    .withColumn('subject_29', split_col_subjects.getItem(29)) \
    .withColumn('subject_30', split_col_subjects.getItem(30)) \
    .withColumn('subject_31', split_col_subjects.getItem(31)) \
    .withColumn('subject_32', split_col_subjects.getItem(32)) \
    .withColumn('subject_33', split_col_subjects.getItem(33)) \
    .withColumn('subject_34', split_col_subjects.getItem(34)) \
    .withColumn('subject_35', split_col_subjects.getItem(35)) \
    .withColumn('subject_36', split_col_subjects.getItem(36)) \
    .withColumn('subject_37', split_col_subjects.getItem(37)) \
    .withColumn('subject_38', split_col_subjects.getItem(38)) \
    .withColumn('subject_39', split_col_subjects.getItem(39)) \
    .withColumn('subject_40', split_col_subjects.getItem(40)) \
    .withColumn('subject_41', split_col_subjects.getItem(41)) \
    .withColumn('subject_42', split_col_subjects.getItem(42)) \
    .withColumn('subject_43', split_col_subjects.getItem(43)) \
    .withColumn('subject_44', split_col_subjects.getItem(44)) \
    .withColumn('subject_45', split_col_subjects.getItem(45)) \
    .withColumn('subject_46', split_col_subjects.getItem(46)) \
    .withColumn('subject_47', split_col_subjects.getItem(47)) \
    .withColumn('subject_48', split_col_subjects.getItem(48)) \
    .withColumn('subject_49', split_col_subjects.getItem(49)) \
    .withColumn('subject_50', split_col_subjects.getItem(50)) \
    .withColumn('subject_51', split_col_subjects.getItem(51)) \
    .withColumn('subject_52', split_col_subjects.getItem(52)) \
    .withColumn('subject_53', split_col_subjects.getItem(53)) \
    .withColumn('subject_54', split_col_subjects.getItem(54)) \
    .withColumn('subject_55', split_col_subjects.getItem(55)) \
    .withColumn('subject_56', split_col_subjects.getItem(56)) \
    .withColumn('subject_57', split_col_subjects.getItem(57)) \
    .withColumn('subject_58', split_col_subjects.getItem(58)) \
    .withColumn('subject_59', split_col_subjects.getItem(59)) \
    .withColumn('subject_60', split_col_subjects.getItem(60)) 

df4.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------+---------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|            Subjects|         ID_Material|           subject_0|           subject_1|           subject_4|           subject_3|subject_5|subject_6|subject_7|subject_8|subject_9|subject_10|subject_11|subject_12|subject_13|subject_14|subject_15|subject_16|subject

In [29]:
count_non_null = df4.select(col('subject_55')).na.drop().count()

print(count_non_null)

2


In [30]:
df4.select('ID_Material','subject_55').na.drop().show(2,False)


+----------------------------------------------------------------+-----------------------------------+
|ID_Material                                                     |subject_55                         |
+----------------------------------------------------------------+-----------------------------------+
|a0b866e2f55042bbca93b13e8af2d63a1b92eb33bedfeaae94e2abe486cf53ec| Puerto Rico Description and travel|
+----------------------------------------------------------------+-----------------------------------+
only showing top 1 row



In [31]:
filtered_df = df4.filter(col('ID_Material') == 'a0b866e2f55042bbca93b13e8af2d63a1b92eb33bedfeaae94e2abe486cf53ec')

# exibe o dataframe filtrado
filtered_df.show(1,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# VERIFICAR DUPLICATAS DEMAIS TABELAS

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.functions.split.html

https://sparkbyexamples.com/pyspark/pyspark-split-dataframe-column-into-multiple-columns/

https://www.projectpro.io/recipes/define-split-function-pyspark

https://www.datasciencemadesimple.com/string-split-of-the-columns-in-pyspark/